#Задание

На вебинаре мы говорили что долгое время CNN и RNN архитектуры были конурируещими выяснить какая архитектура больше подходит для задачи сантимент анализа на данных с вебинара
1. построить свёрточные архитектуры
2. построить различные архитектуры с RNN
3. построить совместные архитектуры CNN -> RNN и (RNN -> CNN)
4. сдлать выводы что получилось лучше

In [106]:
import matplotlib.pyplot as plt

In [107]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [108]:
path = '/content/drive/MyDrive/Colab Notebooks/GB/Introduction to Natural Language Processing/lesson_8/'

In [109]:
!pip install stop_words

In [110]:
!pip install pymorphy2
from pymorphy2 import MorphAnalyzer

In [111]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv(f"{path}/data/train.csv")
df_test = pd.read_csv(f"{path}/data/test.csv")
df_val = pd.read_csv(f"{path}/data/val.csv")

In [112]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [113]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [114]:
df_train.loc[0:2]

,id,text,class
0,0,alisachachkaич уезжаааааааать ❤ тожена хотеть ...,0
1,1,rt galyginvadim ребята девчата кино любовь зав...,1
2,2,rt artemklyushin ктоненавидеть пробка ретвит rt,0


In [160]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking, RepeatVector
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from tensorflow.keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping 
from tensorflow.keras import layers

In [116]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [117]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)

In [118]:
word_count, training_length

(258108, 27)

In [119]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [140]:
result = {'name': list(), 'loss': list(), 'accuracy': list()}

#Общие параметры

In [121]:
batch_size = 512
epochs=10
validation_split = 0.1
verbose=1
early_stopping=EarlyStopping(monitor='val_loss')

#SimpleRNN

In [141]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [142]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 28s 84ms/step - loss: 0.5644 - accuracy: 0.6941 - val_loss: 0.4928 - val_accuracy: 0.7546


In [143]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 13ms/step - loss: 0.5188 - accuracy: 0.7411


Test score: 0.5187587141990662
Test accuracy: 0.7410836219787598


In [144]:
result['name'].append('SimpleRNN')
result['loss'].append(score[0])
result['accuracy'].append(score[1])

#LSTM

Какие проблемы у рекурентных сетей?

- затухают градиенты
- медленно, нужно всегда дойти до конца

Как решить? -> LSTM


<img src="images/lstm.png">


https://colah.github.io/posts/2015-08-Understanding-LSTMs/


Давайте, кратко посмотрим как это работает:


<img src="images/LSTMMaths.png">

In [145]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 54s 163ms/step - loss: 0.5565 - accuracy: 0.7031 - val_loss: 0.4944 - val_accuracy: 0.7515


In [146]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 25ms/step - loss: 0.5186 - accuracy: 0.7402


Test score: 0.5185980200767517
Test accuracy: 0.7402459979057312


In [147]:
result['name'].append('LSTM')
result['loss'].append(score[0])
result['accuracy'].append(score[1])

#GRU

Какие проблемы:

- вычислительно сложно -> медленнее
- на очень длинных последовательностях все равно затухает градиент


Зачем платить больше - уберем некоторые врата (точнее совместим) -> ускоримся, уменьшим число параметров -> GRU


<img src="images/gru.png">


GRU Math


<img src="images/GRUMath.png">


In [148]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 48s 145ms/step - loss: 0.5548 - accuracy: 0.7066 - val_loss: 0.4985 - val_accuracy: 0.7492


In [130]:
# plt.figure(figsize=(20,12))
# plt.plot(history.history['accuracy'], '-.', label = f'accuracy', color='m')
# plt.plot(history.history['val_accuracy'], '-', label = f'val_accuracy', color='g')
        
# plt.legend()
# plt.grid('On')
# plt.title('Accuracy of epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.show()

In [149]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 22ms/step - loss: 0.5277 - accuracy: 0.7341


Test score: 0.5276625752449036
Test accuracy: 0.7341180443763733


In [150]:
result['name'].append('GRU')
result['loss'].append(score[0])
result['accuracy'].append(score[1])

# CNN

In [151]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30))

model.add(Conv1D(256, 3, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [152]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])


Epoch 1/10
319/319 [==============================] - 33s 101ms/step - loss: 0.5448 - accuracy: 0.5076 - val_loss: 0.4850 - val_accuracy: 0.5049


In [153]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 14ms/step - loss: 0.5160 - accuracy: 0.5047


Test score: 0.5159980654716492
Test accuracy: 0.5047392249107361


In [154]:
result['name'].append('CNN')
result['loss'].append(score[0])
result['accuracy'].append(score[1])

#Add Dropout

In [161]:
model_5 = Sequential([
    layers.Embedding(input_dim=word_count, 
                     input_length=training_length, output_dim=30),
    layers.Dropout(0.2),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPool1D(),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

In [162]:
model_5.compile(optimizer='RMSprop',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

In [164]:
history_5 = model_5.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 20s 58ms/step - loss: 0.5711 - accuracy: 0.6903 - val_loss: 0.5081 - val_accuracy: 0.7428


In [165]:
score = model_5.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

result['name'].append('Add Dropout')
result['loss'].append(score[0])
result['accuracy'].append(score[1])

45/45 [==============================] - 0s 11ms/step - loss: 0.5278 - accuracy: 0.7290


Test score: 0.5277639627456665
Test accuracy: 0.7289600372314453


#CNN + RNN

In [189]:
model = Sequential([
    layers.Embedding(input_dim=word_count, 
                     input_length=training_length, output_dim=30),
    layers.Dropout(0.2),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPool1D(),

    layers.RepeatVector(1),
    layers.LSTM(64, recurrent_dropout=0.2),
    
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='RMSprop',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

In [190]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 23s 64ms/step - loss: 0.5799 - accuracy: 0.6802 - val_loss: 0.5112 - val_accuracy: 0.7443


In [191]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 12ms/step - loss: 0.5413 - accuracy: 0.7167


Test score: 0.5413198471069336
Test accuracy: 0.7167041301727295


In [192]:
result['name'].append('CNN + RNN')
result['loss'].append(score[0])
result['accuracy'].append(score[1])

#GRU + CNN

In [199]:
model = Sequential([
    layers.Embedding(input_dim=word_count, 
                     input_length=training_length, output_dim=30),
    layers.Dropout(0.2),
    layers.Conv1D(128, 5, activation='relu'),
    
    layers.GRU(64, recurrent_dropout=0.2),
    layers.RepeatVector(1),
    layers.GlobalMaxPool1D(),
    
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])

In [200]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=verbose,
                    validation_split=validation_split,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 62s 189ms/step - loss: 0.5529 - accuracy: 0.7047 - val_loss: 0.4888 - val_accuracy: 0.7581


In [201]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 35ms/step - loss: 0.5194 - accuracy: 0.7391


Test score: 0.5194015502929688
Test accuracy: 0.7391438484191895


In [202]:
result['name'].append('GRU + CNN optimizer=adam')
result['loss'].append(score[0])
result['accuracy'].append(score[1])

#Result

In [203]:
df_result = pd.DataFrame(result)
df_result

,name,loss,accuracy
0,SimpleRNN,0.518759,0.741084
1,LSTM,0.518598,0.740246
2,GRU,0.527663,0.734118
3,CNN,0.515998,0.504739
4,Add Dropout,0.527764,0.728960
5,CNN + RNN,0.541320,0.716704
6,GRU + CNN,0.522588,0.732619
7,GRU + CNN optimizer=adam,0.519402,0.739144


#Ответ:
Лучший рез-т показала SimpleRNN - 0.741084.

3 подхода:

<img src="images/RNNCompar.png">


Как регуляризовать?
- дропаут
- рекурентный дропаут


<img src="images/Dropouts.png">